In [14]:
import pandas as pd

## Model Training

In [15]:
df = pd.read_csv('./data/gemstone.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [16]:
#Drop the id column
df=df.drop(labels=['id'],axis=1)

In [17]:
#Seperate the independent and dependent features
X = df.drop(labels=['price'],axis=1)
Y = df[['price']]

In [18]:
Y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
30094,5816
30095,6334
30096,12765
30097,1970


In [19]:
#Define which columns should be ordinal-encoded and which should be scaled
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [20]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [21]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [22]:
#from sklearn.impute import SimpleImputer #Handling missing values
#SimpleImputer is a univariate imputer for completing missing values with simple strategies(Google it for more information)
#SimpleImputer is used to replace missing values using descriptive statistic(mean,mode or most_frequent)
#For categorical we can replace with only most_frequent, for numerical we can replace with ay of the three types
#from sklearn.preprocessing import StandardScaler #Handling Feature Scaling
#from sklearn.preprocessing import OrdinalEncoder 
#Ordinal encoding automates the way to replace the categorical values with numeric values
#Simply whereever categorical feature values have ranks we can replace that values using ordinal encoding
#OneHotEncoder for categorical feature values which doesn't have ranks or for nominal type data
#search for ordinaencoder in google for more info
##pipelines -> combinig multiple steps

In [23]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)

# Categorigal Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
    ('scaler',StandardScaler())
    ]

)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])

In [24]:
#TrainTest split
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)

In [25]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [26]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,1.576454,-0.658445,-1.159064,1.542034,1.574490,1.482309,0.867896,-0.938382,0.005035
1,-0.199028,1.829186,1.451584,-0.106088,-0.075345,0.063496,-2.152371,-0.938382,-0.657994
2,2.710195,-0.382042,1.451584,2.240695,2.196559,2.148283,-0.138860,1.534046,-1.321024
3,-1.033291,-2.316866,1.451584,-1.198864,-1.193266,-1.326362,-0.138860,0.915939,0.668065
4,0.228799,-3.422479,0.929455,0.520915,0.501646,0.121407,-0.138860,-0.320275,0.005035


In [27]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [28]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [29]:
regression.coef_

array([[ 6456.94337958,   -38.90331236,   -43.97092791, -1828.68555201,
          481.77147806,  -902.35082285,    86.72987409,  -459.23915904,
          665.77730961]])

In [30]:
regression.intercept_

array([4002.94313921])

In [32]:
import numpy as np
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square=r2_score(true,predicted)
    return mae,rmse,r2_square

In [36]:
#Train multiple models

models={'LinearRegression':LinearRegression(),
        'Lasso':Lasso(),
        'Ridge':Ridge(),
        'ElasticNet':ElasticNet()
        
        }

model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #make predictions
    y_pred=model.predict(X_test)

    mae,rmse,r2_square=evaluate_model(y_test,y_pred)

    
    model_list.append(list(models.keys())[i])
    print(model_list[i])
    
    print("Model Training performance")
    print("MAE:",mae)
    print("RMSE:",rmse)
    print("R2_SQUARE:",r2_square*100)
    r2_list.append(r2_square)

    print("="*35)
    print('\n')





LinearRegression
Model Training performance
MAE: 681.3962435860946
RMSE: 1032.126350272546
R2_SQUARE: 93.51944283293591


Lasso
Model Training performance
MAE: 681.8376892717043
RMSE: 1031.510493843277
R2_SQUARE: 93.52717425473098


Ridge
Model Training performance
MAE: 681.5103436209681
RMSE: 1032.0626332357822
R2_SQUARE: 93.52024294651414


ElasticNet
Model Training performance
MAE: 1061.5207388018864
RMSE: 1527.9445374585132
R2_SQUARE: 85.79759820977243




In [34]:
print(model_list)
print(r2_list)

['LinearRegression', 'Lasso', 'Ridge', 'ElasticNet']
[0.9351944283293591, 0.9352717425473098, 0.9352024294651414, 0.8579759820977243]
